In [1]:
from qiskit import QuantumCircuit, Aer, execute
from qiskit.visualization import array_to_latex
from qiskit_transpiler import get_pm
from math import pi
from qiskit import IBMQ
from qiskit.providers import ibmq

In [5]:
from qiskit.pulse import DriveChannel, GaussianSquare, ShiftPhase

def get_n_link(backend):
    configuration = backend.configuration()
    defaults = backend.defaults()
    n = configuration.num_qubits

    connected_pairs_cnot = configuration.coupling_map
    instruction_schedule_map = defaults.instruction_schedule_map

    def get_control_qubit(q1, q2):  # Control performs Z
        try:
            cx_sched = instruction_schedule_map.get("cx", qubits=(q1, q2))
        except:
            cx_sched = instruction_schedule_map.get("ecr", qubits=(q1, q2))
        supported = False
        for time, inst in cx_sched.instructions:
            if isinstance(inst.channel, DriveChannel) and not isinstance(inst, ShiftPhase):
                if isinstance(inst.pulse, GaussianSquare):
                    target = inst.channel.index
                    control = q1 if target == q2 else q2
                    supported = True
        if not supported:
            raise ValueError("This machine is not supported!")
        return control

    link = []
    for item in connected_pairs_cnot:
        if get_control_qubit(item[0], item[1]) == item[0]:
            link.append(item)

    return n, link

def make_circuit(backend,line,ins,params,t):
    n, link = get_n_link(backend)
    circ = QuantumCircuit(n)
    if line < n:
        if ins == 0:
            q = line
            circ.rz(-params[1], q)
            circ.rx(2 * params[0] * t, q)
            circ.rz(params[1], q)
        else:
            q = line
            lamb = 2 * params[0] * t
            circ.rz(lamb, q)
    else:
        # print(line)
        (q0, q1) = link[line - n]
        theta = 2 * params[0] * t
        if ins == 0:
            circ.rzx(theta, q0, q1)
        elif ins == 1:
            circ.rxx(theta, q0, q1)
        elif ins == 2:
            circ.ryy(theta, q0, q1)
        else:
            circ.rzz(theta, q0, q1)
    return circ

In [3]:
from qiskit_ibm_provider import IBMProvider

with open('../../ibm_API_key','r') as file:
    token = file.readline()
#IBMProvider.save_account(token,instance='ibm-q-ncsu/nc-state/quantum-compiler',overwrite=True)
provider = IBMQ.enable_account(token, hub="ibm-q-ncsu", group="nc-state", project="quantum-compiler")
#provider = IBMProvider()

In [4]:
from qiskit_transpiler import get_pm
backend = provider.get_backend('ibmq_mumbai')
pm = get_pm(backend)

In [6]:
params = [pi/2,pi/2]
t = 0.5

In [7]:
circuits_0 = []
circuits_1 = []

for i in range(27):
    circuits_0.append(pm.run(make_circuit(backend,i,0,params,t)))
    circuits_1.append(pm.run(make_circuit(backend,i,1,params,t)))

In [8]:
from qiskit import transpile as transpile_qiskit
transpiled_circuits_0 = transpile_qiskit(circuits_0,backend)
transpiled_circuits_1 = transpile_qiskit(circuits_1,backend)

In [9]:
from qiskit_aer.noise import NoiseModel
simulator = provider.get_backend("ibmq_qasm_simulator")
noise_model = NoiseModel.from_backend(backend).to_dict()
simulator.options.update_options(noise_model=noise_model)

In [10]:
job = execute(transpiled_circuits_0, shots=1024, backend=simulator)